In [2]:
import pandas as pd

In [3]:
df1 = pd.read_csv('2023_data/df1.csv', encoding='euc-kr')
df2 = pd.read_csv('2023_data/df2.csv', encoding='euc-kr')
df3 = pd.read_csv('2023_data/df3.csv', encoding='euc-kr')
df4 = pd.read_csv('2023_data/df4.csv', encoding='euc-kr')
df5 = pd.read_csv('2023_data/df5.csv', encoding='euc-kr')
df6 = pd.read_csv('2023_data/df6.csv', encoding='euc-kr')

In [5]:
df1.head()

,성별,나이,이동인구(합),행정동명
0,F,20,39.07,서울특별시 종로구 사직동
1,F,20,*,서울특별시 종로구 사직동
2,F,20,*,서울특별시 종로구 사직동
3,F,25,*,서울특별시 종로구 사직동
4,F,25,4.08,서울특별시 종로구 사직동


In [6]:
df2.head()

,성별,나이,이동인구(합),행정동명
0,F,20,*,서울특별시 종로구 사직동
1,F,20,*,서울특별시 종로구 사직동
2,F,20,*,서울특별시 종로구 사직동
3,F,25,6.19,서울특별시 종로구 사직동
4,F,25,*,서울특별시 종로구 사직동


In [7]:
import numpy as np

In [8]:
dataframes = [df1, df2, df3, df4, df5, df6]

In [40]:
months = [1, 2, 3, 4, 5, 6]

In [41]:
updated_dataframes = []

In [ ]:
for i, df in enumerate(dataframes):
    # 이동인구 열의 *을 2로 대체하고 float 형으로 변환
    df['이동인구(합)'] = df['이동인구(합)'].replace('*', 2).astype(float)
    
    # 연월 열 추가 (연도는 예시로 2023년으로 가정)
    df['연월'] = f'2023-{months[i]:02d}'

    # 나이를 10살 단위로 변환하되, 60대 이상은 하나의 그룹으로 묶음
    df['나이대'] = df['나이'].apply(lambda x: 60 if x >= 60 else (x // 10) * 10)
    
    # 경기도와 서울특별시에 해당하는 행만 필터링
    df = df[df['행정동명'].str.contains('경기도|서울특별시')]

    # 행정동별 남녀 비율 계산
    gender_ratio = df.groupby(['행정동명', '성별']).size().unstack().fillna(0)
    gender_ratio['남자 비율'] = gender_ratio.get('M', 0) / gender_ratio.sum(axis=1)
    gender_ratio['여자 비율'] = gender_ratio.get('F', 0) / gender_ratio.sum(axis=1)

    # 행정동별 10살 단위 나이대 비율 계산, 60대 이상은 '60대 이상'으로 묶음
    age_ratio = df.groupby(['행정동명', '나이대']).size().unstack().fillna(0)
    age_ratio = age_ratio.div(age_ratio.sum(axis=1), axis=0)

    # 데이터프레임으로 결과 정리
    result_df = pd.DataFrame({
        '연월': df['연월'].iloc[0],
        '남자 비율': gender_ratio['남자 비율'],
        '여자 비율': gender_ratio['여자 비율']
    })

    # 10살 단위 나이대 비율 열 추가 (0대, 10대, ..., 50대, 60대 이상)
    for age in range(0, 60, 10):
        age_column_name = f'{age}대 비율'
        result_df[age_column_name] = age_ratio.get(age, 0)  # 값이 없으면 0으로 채움
    # 60대 이상 비율 열 추가
    result_df['60대 이상 비율'] = age_ratio.loc[:, 60] if 60 in age_ratio.columns else 0

    # 결과를 리스트에 추가
    updated_dataframes.append(result_df)

In [ ]:
final_result = pd.concat(updated_dataframes)

In [ ]:
final_result.to_csv('2023_01_06.csv', index=True)

In [ ]:
final_data = pd.read_csv('2023_01_06.csv')

In [ ]:
final_data.head()